In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [18]:
# Load the input data
mol_train = pd.read_csv('data/mol_train.csv.gz')
mol_valid = pd.read_csv('data/mol_valid.csv.gz')
mol_test = pd.read_csv('data/mol_test.csv.gz')

smiles_train = pd.read_csv('data/smiles_train.csv.gz', index_col=0)
smiles_valid = pd.read_csv('data/smiles_valid.csv.gz', index_col=0)
smiles_test = pd.read_csv('data/smiles_test.csv.gz', index_col=0)

In [7]:
from keras.models import load_model
from nfp import custom_layers
from nfp.preprocessing import GraphSequence
import warnings
import pickle
from tqdm import tqdm

In [8]:
props = ['gap', 'homo', 'lumo', 'spectral_overlap', 'homo_extrapolated',
         'lumo_extrapolated', 'gap_extrapolated', 'optical_lumo_extrapolated']

def mae(true, pred):
    return np.nanmean(np.abs(true - pred), 0)

In [9]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = load_model('b3lyp_2D_noatom_bn_fixed/best_model.hdf5',
                       custom_objects=custom_layers)
    
with open('b3lyp_2D_noatom_bn_fixed/preprocessor.p', 'rb') as f:
    preprocessor = pickle.load(f)
    
with open('b3lyp_2D_noatom_bn_fixed/y_scaler.p', 'rb') as f:
    y_scaler = pickle.load(f)

inputs_test = preprocessor.predict(smiles_test.smile)
test_inputs_seq = GraphSequence(inputs_test, shuffle=False, final_batch=True, batch_size=128)

100%|██████████| 5000/5000 [00:10<00:00, 477.93it/s]


In [10]:
inputs_train = preprocessor.predict(smiles_train.smile)

100%|██████████| 80823/80823 [03:42<00:00, 363.48it/s]


In [19]:
max((item['n_bond'] for item in inputs_train))

424

In [ ]:
p

In [40]:
y_pred_test = y_scaler.inverse_transform(
    model.predict_generator(test_inputs_seq, verbose=True))

y_true = smiles_test.set_index('smile').values
mae_2d = pd.Series({props[i]: mae(y_pred_test[:, i], y_true[:, i]) for i in range(8)},
                     name='b3lyp_2D_noatom_bn_fixed')

mae_2d = mae_2d * 1000
mae_2d['spectral_overlap'] /= 1000
mae_2d.round(1)

40/40 [==============================] - 5s 134ms/step


gap                           37.2
homo                          32.2
lumo                          32.1
spectral_overlap             158.2
homo_extrapolated             49.6
lumo_extrapolated             48.4
gap_extrapolated              56.7
optical_lumo_extrapolated     46.1
Name: b3lyp_2D_noatom_bn_fixed, dtype: float64

In [35]:
y_multitarget = pd.DataFrame(y_pred_test, columns=props)

In [9]:
st_predictions = []

for prop in props:
    print(prop)
    
    model_name = 'b3lyp_2d_{}'.format(prop)

    with open('{}/preprocessor.p'.format(model_name), 'rb') as f:
        preprocessor = pickle.load(f)
        
    with open('{}/y_scaler.p'.format(model_name), 'rb') as f:
        y_scaler = pickle.load(f)

    inputs_test = preprocessor.predict(smiles_test.smile)
    test_inputs_seq = GraphSequence(inputs_test, shuffle=False, final_batch=True, batch_size=128)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = load_model('{}/best_model.hdf5'.format(model_name),
                           custom_objects=custom_layers)

    y_pred_test = y_scaler.inverse_transform(
        model.predict_generator(test_inputs_seq, verbose=True))
    st_predictions += [y_pred_test]

  1%|          | 62/5000 [00:00<00:16, 303.58it/s]

gap


100%|██████████| 5000/5000 [00:11<00:00, 444.43it/s]


40/40 [==============================] - 5s 134ms/step


  1%|          | 28/5000 [00:00<00:17, 279.44it/s]

...

40/40 [==============================] - 6s 150ms/step


  0%|          | 25/5000 [00:00<00:20, 244.09it/s]

optical_lumo_extrapolated


100%|██████████| 5000/5000 [00:10<00:00, 497.29it/s]


40/40 [==============================] - 6s 145ms/step


In [39]:
y_2d_st = pd.DataFrame([i.flatten() for i in st_predictions], index=props).T

mae_2d_st = pd.Series({props[i]: mae(smiles_test[props].values[:, i], y_2d_st.values[:, i]) for i in range(8)},
                         name='b3lyp_2d_st')

mae_2d_st = mae_2d_st * 1000
mae_2d_st['spectral_overlap'] /= 1000
mae_2d_st.round(1)

gap                           36.9
homo                          32.1
lumo                          27.9
spectral_overlap             149.3
homo_extrapolated             49.1
lumo_extrapolated             47.8
gap_extrapolated              57.1
optical_lumo_extrapolated     47.8
Name: b3lyp_2d_st, dtype: float64

In [13]:
def rbf_expansion(distances, mu=0, delta=0.2, kmax=150):
    k = np.arange(0, kmax)
    logits = -(np.atleast_2d(distances).T - (-mu + delta * k))**2 / delta
    return np.exp(logits)

def precalc_rbfs(inputs):

    for item in tqdm(inputs):

        item['distance_rbf'] = rbf_expansion(item['distance'])
        del item['distance']

    return inputs

In [14]:
from rdkit.Chem import MolFromMolBlock
from nfp.preprocessing import RobustNanScaler

schnet_predictions = []
for prop in props:
    print(prop)
    
    model_name = 'b3lyp_schnet2_{}'.format(prop)

    with open('{}/schnet_preprocessor.p'.format(model_name), 'rb') as f:
        schnet_preprocessor = pickle.load(f)

    inputs_test = schnet_preprocessor.predict((MolFromMolBlock(mol) for _, mol in mol_test.mol.iteritems()))
    rbf_inputs_test = precalc_rbfs(list(inputs_test))
    rbf_input_seq = GraphSequence(rbf_inputs_test, shuffle=False, batch_size=32)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = load_model('{}/best_model.hdf5'.format(model_name),
                           custom_objects=custom_layers)

    itrain = mol_train[mol_train[prop].notna()]

    # Rescale Y matrix
    y_train_raw = itrain[[prop]].values

    y_scaler = RobustNanScaler()
    y_scaler.fit(y_train_raw)

    y_pred = model.predict_generator(rbf_input_seq, verbose=True)
    schnet_predictions += [y_scaler.inverse_transform(y_pred)]

15it [00:00, 133.71it/s]

gap


5000it [00:21, 235.53it/s]
100%|██████████| 5000/5000 [00:14<00:00, 343.83it/s]


157/157 [==============================] - 15s 94ms/step


0it [00:00, ?it/s]

...

157/157 [==============================] - 14s 90ms/step


0it [00:00, ?it/s]

optical_lumo_extrapolated


5000it [00:22, 221.40it/s]
100%|██████████| 5000/5000 [00:13<00:00, 363.97it/s]


157/157 [==============================] - 14s 88ms/step


In [15]:
y_schnet = pd.DataFrame([i.flatten() for i in schnet_predictions], index=props).T

In [41]:
mae_schnet = pd.Series({props[i]: mae(mol_test[props].values[:, i], y_schnet.values[:, i]) for i in range(8)},
                         name='b3lyp_schnet')
mae_schnet = mae_schnet * 1000
mae_schnet['spectral_overlap'] /= 1000
mae_schnet.round(1)

gap                          32.7
homo                         27.0
lumo                         24.8
spectral_overlap             96.6
homo_extrapolated            56.9
lumo_extrapolated            56.8
gap_extrapolated             69.8
optical_lumo_extrapolated    57.2
Name: b3lyp_schnet, dtype: float64

# Predictions using UFF reoptimized molecules

In [17]:
mol_test_uff = pd.read_csv('data/mol_test_uff.csv.gz')

In [23]:
schnet_predictions_uff = []
for prop in props:
    print(prop)
    
    model_name = 'b3lyp_schnet2_{}'.format(prop)

    with open('{}/schnet_preprocessor.p'.format(model_name), 'rb') as f:
        schnet_preprocessor = pickle.load(f)

    inputs_test = schnet_preprocessor.predict((MolFromMolBlock(mol) for _, mol in mol_test_uff.molUFF.iteritems()))
    rbf_inputs_test = precalc_rbfs(list(inputs_test))
    rbf_input_seq = GraphSequence(rbf_inputs_test, shuffle=False, batch_size=32)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = load_model('{}/best_model.hdf5'.format(model_name),
                           custom_objects=custom_layers)

    itrain = mol_train[mol_train[prop].notna()]

    # Rescale Y matrix
    y_train_raw = itrain[[prop]].values

    y_scaler = RobustNanScaler()
    y_scaler.fit(y_train_raw)

    y_pred = model.predict_generator(rbf_input_seq, verbose=True)
    schnet_predictions_uff += [y_scaler.inverse_transform(y_pred)]

14it [00:00, 132.58it/s]

gap


5000it [00:21, 228.04it/s]
100%|██████████| 5000/5000 [00:14<00:00, 336.39it/s]


157/157 [==============================] - 17s 108ms/step


0it [00:00, ?it/s]

...

157/157 [==============================] - 17s 107ms/step


0it [00:00, ?it/s]

optical_lumo_extrapolated


5000it [00:21, 227.70it/s]
100%|██████████| 5000/5000 [00:15<00:00, 317.02it/s]


157/157 [==============================] - 18s 112ms/step


In [25]:
y_schnet_uff = pd.DataFrame([i.flatten() for i in schnet_predictions_uff], index=props).T

In [45]:
mae_schnet_uff = pd.Series({props[i]: mae(mol_test[props].values[:, i], schnet_predictions_uff[i].flatten()) for i in range(8)},
                         name='b3lyp_schnet_uff')

mae_schnet_uff = mae_schnet_uff * 1000
mae_schnet_uff['spectral_overlap'] /= 1000
mae_schnet_uff.round(0)

gap                           616.0
homo                          353.0
lumo                          517.0
spectral_overlap             1543.0
homo_extrapolated             482.0
lumo_extrapolated             472.0
gap_extrapolated              633.0
optical_lumo_extrapolated     457.0
Name: b3lyp_schnet_uff, dtype: float64

In [36]:
model_predictions = {
    '2d_multitarget': y_multitarget,
    '2d_singletarget': y_2d_st,
    'schnet': y_schnet,
    'schnet_uff': y_schnet_uff,
}

with open('model_predictions.p', 'wb') as f:
    pickle.dump(model_predictions, f)